In [4]:

import numpy as np
import torch
#RockPool imports
from rockpool.devices import xylo as x
from rockpool.transform import quantize_methods as q
from rockpool.devices.xylo import config_from_specification
from rockpool.devices.xylo import XyloSim
from rockpool.nn.modules import LinearTorch, LIFTorch, ExpSynTorch, LIFBitshiftTorch, aLIFTorch
from rockpool.nn.combinators import Sequential
from rockpool.parameters import Constant


ImportError: cannot import name 'LIFBaseTorch' from 'rockpool.nn.modules' (/home/mina/tools/rockpool/rockpool/nn/modules/__init__.py)

In [2]:
def ahp_snn(Nin=16, Nhid1=256, Nout=2, tau_mem = Constant(0.02), tau_syn = Constant(0.02), dt=0.001, wahp = -0.8):

    input_layer = LinearTorch((Nin, Nhid1), has_bias=False)
   
    hid_layer1 = aLIFTorch((Nhid1, Nhid1), tau_syn=tau_syn, tau_mem=tau_mem, threshold=Constant(1.0), 
    max_spikes_per_dt = 31,learning_window=0.5, bias=Constant(0.0), has_ahp = True, w_ahp = Constant(torch.ones(Nhid1,1)*wahp), dt=dt)
    
    lin_out = LinearTorch((Nhid1, Nout), has_bias = False)

    spk_out = LIFTorch((Nout, Nout), tau_syn=tau_syn, tau_mem=tau_mem, threshold=Constant(1e6), 
    max_spikes_per_dt = 1,learning_window=0.5, bias=Constant(0.0),dt=dt)

    
    model = Sequential(input_layer, hid_layer1, lin_out, spk_out)
    
    return model 

In [3]:
# trained model in RockPool
# trained_model =  'models/' + 'wahp=-0_8_HeyNet_ahpLif_onelayer_model.json'
trained_model = 'models/for_mapper_wahp=-0_8_HeyNet_ahpLif_onelayer_model_batchsize=256_lr=0_0001_acc=0_572.json'
# building an instance of architecture and loading trained model into it
model = ahp_snn()
print(model)
model.load(trained_model)
g = model.as_graph()
spec = x.mapper(g, weight_dtype='float', threshold_dtype='float', 
                   dash_dtype='float')

TorchSequential  with shape (16, 2) {
    LinearTorch '0_LinearTorch' with shape (16, 256)
    aLIFTorch '1_aLIFTorch' with shape (256, 256)
    LinearTorch '2_LinearTorch' with shape (256, 2)
    LIFTorch '3_LIFTorch' with shape (2, 2)
}


In [ ]:
spec